In [2]:
import os
import pandas as pd
import mysql.connector
import plotly.express as px
from plotly import graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

In [4]:
import chart_studio

chart_studio.tools.set_credentials_file(username = os.environ.get('cs_user'), api_key = os.environ.get('cs_key'))

import chart_studio.plotly as py
import chart_studio.tools as tls

In [5]:
usr = os.environ.get('sql_user')
pwd = os.environ.get('sql_pwd')
host = os.environ.get('sql_host')
db = os.environ.get('sql_db')

def query(sql):
    
    cnx = mysql.connector.connect(user=usr, 
                              password=pwd,
                              host=host,
                              database=db,
                                 port=3306)
    
    df = pd.read_sql(sql, cnx)
    cnx.close()
    return df

In [6]:
do_three_query = '''
SELECT num_do, count(*) as count
FROM
(
SELECT u.id, count(DISTINCT oc.objective_id) as num_do
FROM objective_completed oc
LEFT JOIN user u
ON oc.user_id = u.id
WHERE oc.date_created BETWEEN u.date_created AND u.date_created + interval 3 day
AND u.date_created <= now() - interval 14 day
GROUP BY u.id
ORDER BY u.id
) as num_do_table
GROUP BY num_do
'''

do_three = query(do_three_query)

In [7]:
do_three

,num_do,count
0,1,2848
1,2,1003
2,3,469
3,4,87


In [5]:
do_three_ret_query = '''
SELECT num_do, count(*) as count
FROM
(
SELECT u.id, count(DISTINCT oc.objective_id) as num_do
FROM objective_completed oc
LEFT JOIN user u
ON oc.user_id = u.id
WHERE oc.date_created BETWEEN u.date_created AND u.date_created + interval 3 day
AND u.date_created <= now() - interval 14 day
AND u.id IN
(
SELECT ecm.user_id
FROM education_completed_module ecm
LEFT JOIN user u
ON u.id = ecm.user_id
WHERE 
			ecm.date_created BETWEEN u.date_created + interval 7 day AND
															 u.date_created + interval 14 day
AND u.date_created <= now() - interval 14 day
GROUP BY user_id
)
GROUP BY u.id
ORDER BY u.id
) as num_do_table
GROUP BY num_do
'''

do_three_ret = query(do_three_ret_query)

In [8]:
def fulltab(alltab, rettab):
    
    alltab['least_sum'] = alltab.loc[::-1, 'count'].cumsum()[::-1]
    rettab['least_sum'] = rettab.loc[::-1, 'count'].cumsum()[::-1]
    
    combtab = pd.merge(alltab, rettab, on = 'num_do')
    
    combtab = combtab.rename(columns = {"least_sum_x": "all", "least_sum_y": "ret_did"})
    
    combtab['ret_didnt'] = combtab['count_y'].cumsum()
    combtab['ret_didnt'] = combtab['ret_didnt'].shift(1)
    combtab['ret_didnt'][0] = 0
    combtab['ret_didnt'] = combtab['ret_didnt'].astype(int)
    combtab['ret_didnt'] = combtab['ret_didnt'] + 160 
    
    # 160 in the line above refers to the number of users who were retained but didn't complete a single daily
    # objective in their first three days
    
    combtab['prop'] = combtab['ret_did'] / (combtab['all'] + combtab['ret_didnt'])
    
    return combtab

In [9]:
fulltab(do_three, do_three_ret)

<ipython-input-8-40ce4b83ac04>:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,num_do,count_x,all,count_y,ret_did,ret_didnt,prop
0,1,1885,3019,254,806,160,0.253539
1,2,694,1134,274,552,414,0.356589
2,3,360,440,218,278,688,0.246454
3,4,80,80,60,60,906,0.060852


In [14]:
fig_do_three = px.bar(fulltab(do_three, do_three_ret), x = 'num_do', y = 'prop')

fig_do_three.show()

<ipython-input-8-40ce4b83ac04>:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [15]:
py.plot(fig_do_three, name = "Daily Objective Aha Moment", auto_open = True)

'https://plotly.com/~woonggyu.jin/116/'